In [2]:
using CSV, DataFrames
using Dates

In [12]:
basepath = normpath("../../");

In [9]:
start_date = Date(2020, 5, 1)
end_date = Date(2020, 6, 1);

In [10]:
locations = ["AL","AZ","AR","CA","CO","CT","DE","FL","GA","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
sort!(locations);

In [44]:
function load_ihme(;forecast_date::String="2020-06-03")
    forecast_data = CSV.read(joinpath(basepath, "data/forecasts/ihme/$(forecast_date)/forecast.csv"), copycols=true)

    state_list = CSV.read(joinpath(basepath, "data/geography/states.csv"), copycols=true)
    
    ga = state_list[(state_list.abbrev .== "GA"),:]
    ga.state .= "Georgia_two"
    append!(state_list, ga)
    
    filter!(row -> row.location_name in state_list.state, forecast_data)

    state_cvt = Dict(state.state => state.abbrev for state in eachrow(state_list))
    forecast_data.state = [state_cvt[row.location_name] for row in eachrow(forecast_data)]

    sort!(forecast_data, [:state, :date])
    return forecast_data
end

function ihme_filter!(forecast::DataFrame, states::Array{String,1}, start_date::Date, end_date::Date)
    @assert states == sort(states)
    @assert start_date <= end_date
    filter!(row -> row.state in states, forecast)
    filter!(row -> start_date <= row.date <= end_date, forecast)
    return forecast
end

function ihme_forecast(forecast::DataFrame; forecast_type::Symbol=:active, patient_type::Symbol=:regular, bound_type::Symbol=:mean)
    forecast_by_state = groupby(forecast, :state, sort=true)

    col_select = Dict(
        (:admitted, :regular, :mean) => :admis_mean,
        (:admitted, :regular, :lb)   => :admis_lower,
        (:admitted, :regular, :ub)   => :admis_upper,
        (:active, :regular, :mean) => :allbed_mean,
        (:active, :regular, :lb)   => :allbed_lower,
        (:active, :regular, :ub)   => :allbed_upper,
        (:admitted, :icu, :mean) => :newICU_mean,
        (:admitted, :icu, :lb)   => :newICU_lower,
        (:admitted, :icu, :ub)   => :newICU_upper,
        (:active, :icu, :mean) => :ICUbed_mean,
        (:active, :icu, :lb)   => :ICUbed_lower,
        (:active, :icu, :ub)   => :ICUbed_upper,
    )
    col = col_select[(forecast_type, patient_type, bound_type)]

    patients = vcat([f[:,col]' for f in forecast_by_state]...)
    return Float32.(patients)
end;

In [45]:
forecast = load_ihme()
forecast = ihme_filter!(forecast, locations, start_date, end_date)
patients = ihme_forecast(forecast, forecast_type=:admitted, patient_type=:regular, bound_type=:mean);
size(patients)

(48, 32)